In [1]:
import tensorflow as tf
print('tensorflow:' + tf.__version__)
from tensorflow.python.platform import gfile

import numpy as np
import math

import cv2
print('opencv:' +  cv2.__version__)

tensorflow:1.11.0
opencv:3.4.3


### Inference config

In [2]:
IMG_PATH = "./image/"
IMG_NAME = "div2k_crop_1.png"
IMG_CHANNEL = 3
LAYER_GLOBAL = 16
LAYER_LOCAL = 6
SCALE = 2

IMG_OPENCV_NAME = "opencv.png"
IMG_RDN_NAME = "rdn.png"

In [3]:
def im2double(image):
    info = np.iinfo(image.dtype) # Get the data type of the input image
    return image.astype(np.float32) / info.max # Divide all values by the largest possible value in the datatype

def bicubic_sr(input, scale):
    bicubic_output = np.clip(cv2.resize(input.copy(), None, fx=1.0*scale, fy=1.0*scale, interpolation=cv2.INTER_CUBIC),0,1) * 255
    bicubic_output = bicubic_output

    return bicubic_output

### Generate super resolution image by RDN

In [4]:
GRAPH_PB_PATH = './fumizuki_rdn.pb'

sess = tf.Session()

with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
   graph_def = tf.GraphDef()

graph_def.ParseFromString(f.read())
sess.graph.as_default()
tf.import_graph_def(graph_def, name='')

#for i in tf.get_default_graph().get_operations():
#    print(i.name)

In [5]:
image = cv2.imread(IMG_PATH + IMG_NAME).astype(np.uint8)
height, width =  image.shape[:2]

imageDouble = im2double(image.copy())
# ================================
tick1 = cv2.getTickCount()
RDN_output = sess.run("HR_conv_layer:0", feed_dict={"RDN_input:0":np.expand_dims(imageDouble, axis=0)})
tick2 = cv2.getTickCount()
# ================================

RDN_output = np.clip(RDN_output*255, 0, 255)
shape = RDN_output.shape
RDN_upscale = RDN_output.reshape(shape[1], shape[2], shape[3])
cv2.imwrite(IMG_RDN_NAME, RDN_upscale, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

True

### Generate super resolution image by opencv bicubic interpolation

In [6]:
bicubic_upscale = bicubic_sr(imageDouble.copy(), scale=SCALE)
cv2.imwrite(IMG_OPENCV_NAME, bicubic_upscale, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

True

### Display image

In [7]:
pltShow = False

if pltShow:    
    from matplotlib import pyplot as plt
    %matplotlib inline
    import seaborn as sns
    
    im1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im2 = cv2.imread(IMG_OPENCV_NAME)
    im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
    im3 = cv2.imread(IMG_RDN_NAME)
    im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)
    
    plt.figure(num='comparison',figsize=(64,64))

    plt.subplot(3,1,1)
    plt.title('origin image')
    plt.imshow(im1)

    plt.subplot(3,1,2)
    plt.title('bicubic_upscale')
    plt.imshow(im2)

    plt.subplot(3,1,3)
    plt.title("RDN_upscale")
    plt.imshow(im3)

### Process time calculation

In [8]:
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("Inference processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("Inference processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("Inference processed time: " + str(tick) + " ms")

Inference processed time: 2 mins 52 secs 510 ms
